In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('GFM_data.csv', sep = '\t')

In [2]:
import tensorflow as tf # build model
import string # get set of punctuations
import requests # get data file in notebook

In [3]:
#!pip install nltk
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.corpus import wordnet

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
REPLACE_IP_ADDRESS = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
def extract_entities(text):
    names = []
    for sent in nltk.sent_tokenize(text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label'):
                names.append(' '.join(c[0] for c in chunk.leaves()))
    new_text = text
    for name in names:
        if name in text:
            new_text = new_text.replace(name, 'NLP')
    return new_text

def clean_text(x):
    ## removing names
    x = extract_entities(x)
    ## normalizing text by stripping white space and lower casing
    x =  x.lower().strip()
    ## removing urls
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x)
    ## removing phone numbers
    x = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}','',x)
    ## strip all non alphanumeric things
    x = re.sub('\n',' ',x)
    x = re.sub("[^a-zA-Z0-9 #]",'',x)
    x = re.sub("\s+",' ',x)
    text = x.replace('\n', ' ').lower()# lowercase text
    text = REPLACE_IP_ADDRESS.sub('', text) # remove ip address
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    # originally:
    # text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    text = [w for w in text.split() if not w in STOPWORDS]# delete stopwords from text   
    return text

In [4]:
no_nan = df.dropna()
no_nan['Text'] = no_nan['Text'].apply(clean_text)
text = no_nan.get('Text').to_numpy()

C:\Users\sddjl\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
#text # check what's inside

In [6]:
tokens = [] # combine words
for lst in text:
    tokens += lst
len(tokens)

144113

In [7]:
#text = ' '.join(text) # join all array element
#new_text = clean_text(text)
#new_text
length = 51 # learn initial 50 words, predict the next word
lines = [] # lst of all training sequences
tokens = text[0]
sufficiency = 200_000
for i in range(length, len(tokens)):
    seq = tokens[i-length:i]
    line = ' '.join(seq)
    lines.append(line)
    if i > sufficiency:
        break

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LambdaCallback

In [9]:
tokenizer = Tokenizer(oov_token="<OOV>") # OOV adds new words outside the trained words
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines) # all words become numbers

In [10]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1] # 50 first words in X, the last word in y

In [11]:
# one-hot encoding
vocab_size = len(tokenizer.word_index) + 1
y = to_categorical(y, num_classes=vocab_size)

In [12]:
sequence_length = len(X[0]) # Length of each list in X is 50 (might also use X.shape[1])
 
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = sequence_length)) # input dim - vocabsize, output dim = 50
# 100 hidden layers
model.add(LSTM(100, return_sequences=True)) # do twice, return_sequences=True
model.add(LSTM(100)) 
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            4100      
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 82)                8282      
Total params: 163,282
Trainable params: 163,282
Non-trainable params: 0
_________________________________________________________________


In [14]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

        y_predict = model.predict_classes(encoded)

        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [15]:
alltext = df['Text'].str.cat(sep='. ').lower()
#alltext

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(X, y, batch_size=256, epochs=1000)

Train on 41 samples
Epoch 1/1000
41/41 [==============================] - 4s 107ms/sample - loss: 4.4071 - accuracy: 0.0000e+00
Epoch 2/1000
41/41 [==============================] - 0s 2ms/sample - loss: 4.4035 - accuracy: 0.0976
Epoch 3/1000
41/41 [==============================] - 0s 2ms/sample - loss: 4.3994 - accuracy: 0.0976
Epoch 4/1000
41/41 [==============================] - 0s 3ms/sample - loss: 4.3944 - accuracy: 0.0976
Epoch 5/1000
41/41 [==============================] - 0s 3ms/sample - loss: 4.3881 - accuracy: 0.0976
Epoch 6/1000
41/41 [==============================] - 0s 3ms/sample - loss: 4.3797 - accuracy: 0.0976
Epoch 7/1000
41/41 [==============================] - 0s 3ms/sample - loss: 4.3679 - accuracy: 0.0976
Epoch 8/1000
41/41 [==============================] - 0s 3ms/sample - loss: 4.3505 - accuracy: 0.0976
Epoch 9/1000
41/41 [==============================] - 0s 2ms/sample - loss: 4.3228 - accuracy: 0.0976
Epoch 10/1000
41/41 [==============================] - 0

Epoch 81/1000
41/41 [==============================] - 0s 2ms/sample - loss: 2.0597 - accuracy: 0.3171
Epoch 82/1000
41/41 [==============================] - 0s 3ms/sample - loss: 2.0243 - accuracy: 0.3659
Epoch 83/1000
41/41 [==============================] - 0s 5ms/sample - loss: 2.0179 - accuracy: 0.3902
Epoch 84/1000
41/41 [==============================] - 0s 5ms/sample - loss: 2.0397 - accuracy: 0.2439
Epoch 85/1000
41/41 [==============================] - 0s 4ms/sample - loss: 1.9967 - accuracy: 0.3415
Epoch 86/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.9464 - accuracy: 0.3659
Epoch 87/1000
41/41 [==============================] - 0s 4ms/sample - loss: 1.9149 - accuracy: 0.4878
Epoch 88/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.9225 - accuracy: 0.3902
Epoch 89/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.9355 - accuracy: 0.2927
Epoch 90/1000
41/41 [==============================] - 0s 3ms/sample - lo

Epoch 160/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.1555 - accuracy: 0.5854
Epoch 161/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.0565 - accuracy: 0.7073
Epoch 162/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.1824 - accuracy: 0.4146
Epoch 163/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.0451 - accuracy: 0.7561
Epoch 164/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.0443 - accuracy: 0.6829
Epoch 165/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.0832 - accuracy: 0.5366
Epoch 166/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.9926 - accuracy: 0.8293
Epoch 167/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.0141 - accuracy: 0.7561
Epoch 168/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.0222 - accuracy: 0.7073
Epoch 169/1000
41/41 [==============================] - 0s 3ms/s

Epoch 239/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.6850 - accuracy: 0.9268
Epoch 240/1000
41/41 [==============================] - 0s 3ms/sample - loss: 1.1832 - accuracy: 0.2927
Epoch 241/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.6916 - accuracy: 0.8293
Epoch 242/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.9839 - accuracy: 0.3659
Epoch 243/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.6579 - accuracy: 0.9512
Epoch 244/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.9007 - accuracy: 0.4146
Epoch 245/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.6708 - accuracy: 0.9512
Epoch 246/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.7583 - accuracy: 0.6585
Epoch 247/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.6661 - accuracy: 0.9268
Epoch 248/1000
41/41 [==============================] - 0s 2ms/s

Epoch 318/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3275 - accuracy: 0.9756
Epoch 319/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.3246 - accuracy: 0.9756
Epoch 320/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3219 - accuracy: 0.9756
Epoch 321/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3198 - accuracy: 0.9756
Epoch 322/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3177 - accuracy: 0.9756
Epoch 323/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3168 - accuracy: 0.9756
Epoch 324/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3148 - accuracy: 0.9756
Epoch 325/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.3120 - accuracy: 0.9756
Epoch 326/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.3081 - accuracy: 0.9756
Epoch 327/1000
41/41 [==============================] - 0s 3ms/s

...

Epoch 634/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0352 - accuracy: 1.0000
Epoch 635/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0349 - accuracy: 1.0000
Epoch 636/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0347 - accuracy: 1.0000
Epoch 637/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0345 - accuracy: 1.0000
Epoch 638/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0343 - accuracy: 1.0000
Epoch 639/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0342 - accuracy: 1.0000
Epoch 640/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0340 - accuracy: 1.0000
Epoch 641/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0337 - accuracy: 1.0000
Epoch 642/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0335 - accuracy: 1.0000
Epoch 643/1000
41/41 [==============================] - 0s 3ms/s

Epoch 713/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0228 - accuracy: 1.0000
Epoch 714/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0227 - accuracy: 1.0000
Epoch 715/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0225 - accuracy: 1.0000
Epoch 716/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0224 - accuracy: 1.0000
Epoch 717/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0224 - accuracy: 1.0000
Epoch 718/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.0222 - accuracy: 1.0000
Epoch 719/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0221 - accuracy: 1.0000
Epoch 720/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0220 - accuracy: 1.0000
Epoch 721/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0219 - accuracy: 1.0000
Epoch 722/1000
41/41 [==============================] - 0s 3ms/s

Epoch 792/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.0157 - accuracy: 1.0000
Epoch 793/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0156 - accuracy: 1.0000
Epoch 794/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0156 - accuracy: 1.0000
Epoch 795/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.0156 - accuracy: 1.0000
Epoch 796/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.0155 - accuracy: 1.0000
Epoch 797/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0154 - accuracy: 1.0000
Epoch 798/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0153 - accuracy: 1.0000
Epoch 799/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0152 - accuracy: 1.0000
Epoch 800/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0152 - accuracy: 1.0000
Epoch 801/1000
41/41 [==============================] - 0s 3ms/s

Epoch 871/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0114 - accuracy: 1.0000
Epoch 872/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0114 - accuracy: 1.0000
Epoch 873/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.0113 - accuracy: 1.0000
Epoch 874/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0113 - accuracy: 1.0000
Epoch 875/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0112 - accuracy: 1.0000
Epoch 876/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0112 - accuracy: 1.0000
Epoch 877/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0112 - accuracy: 1.0000
Epoch 878/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0111 - accuracy: 1.0000
Epoch 879/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0111 - accuracy: 1.0000
Epoch 880/1000
41/41 [==============================] - 0s 3ms/s

Epoch 950/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0086 - accuracy: 1.0000
Epoch 951/1000
41/41 [==============================] - 0s 2ms/sample - loss: 0.0085 - accuracy: 1.0000
Epoch 952/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0085 - accuracy: 1.0000
Epoch 953/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0085 - accuracy: 1.0000
Epoch 954/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0085 - accuracy: 1.0000
Epoch 955/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0084 - accuracy: 1.0000
Epoch 956/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0084 - accuracy: 1.0000
Epoch 957/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0084 - accuracy: 1.0000
Epoch 958/1000
41/41 [==============================] - 0s 3ms/sample - loss: 0.0084 - accuracy: 1.0000
Epoch 959/1000
41/41 [==============================] - 0s 3ms/s

In [18]:
seed_text=alltext[:len(df['Text'][0])+16]
seq_length = len(X[0])
#seed_text

In [19]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []

    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

        y_predict = model.predict_classes(encoded)

        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        print('--SEED TEXT--')
        print(seed_text)
        seed_text = seed_text + ' ' + predicted_word
        print('--PREDICTED WORDS--', predicted_word)
        print('----------------------------------------------------------')
        text.append(predicted_word)
    return ' '.join(text)

In [20]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n
--PREDICTED WO

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

...

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

--SEED TEXT--
on august 23rd my son was shot multiple times in the back by a kenosha police department officer, after my son broke up an altercation by an unrelated party. the shooting has now left my son critically injured as he fights for his life. the extent of my son’s injuries is unknown, but we remain prayerful as he continues to undergo multiple rounds of operations to save his life.

this fund is established to cover my son’s medical expenses, mental and grief counseling for our family and to assist our family in the days to come, as we continue to seek justice for jacob.  a portion of these proceeds will also be used to benefit my son’s six children, some of whom witnessed this horrific act of violence.

anyone wishing to send cards, letters of encouragement  and/or contributions in the form of a money order or check, may do so by mail at:
 
julia y. jackson
c/o ben crump law, pllc
122 s. calhoun street
tallahassee, fl 32301
attn: adner marcelin
 . 
the navajo n attn attn fl f

'attn attn fl form form form violence act act used used used used used act act wishing letters contributions co street street street tallahassee tallahassee fl tallahassee calhoun calhoun calhoun calhoun street attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn attn tallahassee tallahassee form violence violence used used wishing order 122 mail tallahassee street attn attn attn attn attn contributions act act act form order tallahassee tallahassee tallahassee tallahassee tallahassee tallahassee tallahassee violence violence violence violence violence act act violence violence violence violence'

https://kgptalkie.com/text-generation-using-tensorflow-keras-and-lstm/

https://www.youtube.com/watch?v=VAMKuRAh2nc&t=1607s&ab_channel=KGPTalkie